# Dashboard ETL Process

In [1]:
import csv
import json
import pandas as pd
import numpy as np

In [2]:
# Define filepaths for import
baseLocation = r'C:\Users\billy\group_project'
countriesFilePath = r'C:\Users\billy\group_project\Group_Resources\countries.csv'
cleanedFilePath = r'C:\Users\billy\group_project\ETL\model_dataset_cleaned.csv'
initjsonFilePath = r'C:\Users\billy\group_project\Dashboard\static\js\temp\countryBoundries.json'
jsonOutputFilePath = r'C:\Users\billy\group_project\Dashboard\static\js\temp\countryDataBoundriesOutput.json'

In [3]:
# Load GeoJson data for update
file = open(initjsonFilePath)
countryBoundries = json.load(file)

In [4]:
# Load cleaned dataset to pull data for json file update
df_cleaned = pd.read_csv(cleanedFilePath)
df_cleaned.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,population,gdp_per_capita,dob_converted,age,total_medals,placed
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0,46418269.0,25831.582305,2069-10-17,46,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0,50617045.0,27221.524051,1986-09-23,29,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,35851774.0,43248.529909,1992-05-27,24,1,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0,3554150.0,1848.061804,1991-01-02,25,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0,4595700.0,37807.967276,1990-11-26,25,0,0


In [5]:
# Function for transforming csv dataframe data into elements that can be joined with the JSON file
def defineFeatures(groupFunction, dfInput, grouping, calcColumn, rounding):
    if groupFunction == 'count':
        df_defineFeatures = (dfInput.groupby(grouping)
                            .count()
                            .reset_index()[[grouping, calcColumn]]
                            .set_index(grouping)
                    )
    elif round == 0:
        df_defineFeatures = (dfInput.groupby(grouping)
                                    .agg(groupFunction)
                                    .reset_index()[[grouping, calcColumn]]
                                    .set_index(grouping)
                            )                    
    else:
        df_defineFeatures = (dfInput.groupby(grouping)
                                    .agg(groupFunction)
                                    .reset_index()[[grouping, calcColumn]]
                                    .set_index(grouping).round(rounding)
                            )

    dict_defineFeatures = df_defineFeatures.to_dict(orient='dict')[calcColumn]    
    
    df_defineFeatures['percentile'] = df_defineFeatures[calcColumn].rank(pct=True)
    df_defineFeatures.sort_values('percentile', ascending=False, inplace=True)

    #percentileCol = f'percentile_{calcColumn}'
    df_defineRank = df_defineFeatures.drop(columns=[calcColumn])
    #.rename({'percentile': percentileCol}, axis=1)
    #dict_defineRank = df_defineRank.to_dict(orient='dict')[percentileCol]
    dict_defineRank = df_defineRank.to_dict(orient='dict')['percentile']
    

    return [df_defineFeatures, dict_defineFeatures, dict_defineRank]

In [6]:
# Output list with dataframe and dictionary for required country data
out_totalMedals = defineFeatures('sum', df_cleaned, 'nationality', 'total_medals', 0)
out_countAthlete = defineFeatures('count', df_cleaned, 'nationality', 'name', 0)
out_meanHeight = defineFeatures('mean', df_cleaned, 'nationality', 'height', 4)
out_meanWeight = defineFeatures('mean', df_cleaned, 'nationality', 'weight', 4)
out_meanAge = defineFeatures('mean', df_cleaned, 'nationality', 'age', 4)
out_population = defineFeatures('mean', df_cleaned, 'nationality', 'population', 0)
out_gdp = defineFeatures('mean', df_cleaned, 'nationality', 'gdp_per_capita', 0)

In [7]:
# Parse dictionaries to update JSON file with athlete and country data
dict_totalMedals = out_totalMedals[1]
dict_countAthlete = out_countAthlete[1]
dict_meanHeight = out_meanHeight[1]
dict_meanWeight = out_meanWeight[1]
dict_meanAge = out_meanAge[1]
dict_population = out_population[1]
dict_gdp = out_gdp[1]

In [8]:
# Parse dictionaries to update JSON file with athlete and country rankings
rank_totalMedals = out_totalMedals[2]
rank_countAthlete = out_countAthlete[2]
rank_meanHeight = out_meanHeight[2]
rank_meanWeight = out_meanWeight[2]
rank_meanAge = out_meanAge[2]
rank_population = out_population[2]
rank_gdp = out_gdp[2]

## Normalization of GDP and Medal Counts for Map Layers

In [9]:
# Calculate per capita (millions) medal counts and ranks
df_medalsPerCapitaMill = out_totalMedals[0].merge(out_population[0],
                                                  how='left',
                                                  on='nationality',
                                                  validate='many_to_one').drop(columns=['percentile_x', 'percentile_y'])
df_medalsPerCapitaMill['medalsPerCapita'] = (1000000 
                                             * (df_medalsPerCapitaMill['total_medals']
                                             /df_medalsPerCapitaMill['population']
                                                )
                                             ).round(4)
dict_medalsPerCapitaMill = df_medalsPerCapitaMill.to_dict(orient='dict')['medalsPerCapita']

df_medalsPerCapitaMill['percentile'] = df_medalsPerCapitaMill['medalsPerCapita'].rank(pct=True)
df_medalsPerCapitaMill.sort_values('percentile', ascending=False, inplace=True)

rank_medalsPerCapitaMill = df_medalsPerCapitaMill.to_dict(orient='dict')['percentile']

In [10]:
# Calculate per gdp medal counts
df_medalsPerGdpThous = out_totalMedals[0].merge(out_gdp[0],
                                           how='left',
                                           on='nationality',
                                           validate='many_to_one').drop(columns=['percentile_x', 'percentile_y'])
df_medalsPerGdpThous['medalsPerGdp'] = (1000
                                             * (df_medalsPerGdpThous['total_medals']
                                             /df_medalsPerGdpThous['gdp_per_capita']
                                                )
                                             ).round(4)
dict_medalsPerGdpThous = df_medalsPerGdpThous.to_dict(orient='dict')['medalsPerGdp']

df_medalsPerGdpThous['percentile'] = df_medalsPerGdpThous['medalsPerGdp'].rank(pct=True)
df_medalsPerGdpThous.sort_values('percentile', ascending=False, inplace=True)

rank_medalsPerGdpThous = df_medalsPerGdpThous.to_dict(orient='dict')['percentile']

## Merge Olympic data with GeoJSON data

In [11]:
# Navigate to features in JSON file
dict_features = countryBoundries['features']

In [12]:
# Append JSON dictionary with athlete and country data
for item in dict_features:
    iso = item['properties']['ISO_A3']
    item['properties']['population'] = dict_population.get(iso)
    item['properties']['rank_population'] = rank_population.get(iso)
    item['properties']['gdp_per_capita'] = dict_gdp.get(iso)    
    item['properties']['rank_gdp_per_capita'] = rank_gdp.get(iso) 
    item['properties']['sumMedals'] = dict_totalMedals.get(iso)
    item['properties']['rank_sumMedals'] = rank_totalMedals.get(iso)
    item['properties']['countAthletes'] = dict_countAthlete.get(iso)
    item['properties']['rank_countAthletes'] = rank_countAthlete.get(iso)
    item['properties']['meanAthHeight'] = dict_meanHeight.get(iso)
    item['properties']['rank_meanAthHeight'] = rank_meanHeight.get(iso)
    item['properties']['meanAthWeight'] = dict_meanWeight.get(iso)
    item['properties']['rank_meanAthWeight'] = rank_meanWeight.get(iso)
    item['properties']['meanAthAge'] = dict_meanAge.get(iso)
    item['properties']['rank_meanAthAge'] = rank_meanAge.get(iso)
    item['properties']['MedalsPerCapitaMill'] = dict_medalsPerCapitaMill.get(iso)
    item['properties']['rank_MedalsPerCapitaMill'] = rank_medalsPerCapitaMill.get(iso)
    item['properties']['MedalsPerGdpThous'] = dict_medalsPerGdpThous.get(iso)
    item['properties']['rank_MedalsPerGdpThous'] = rank_medalsPerGdpThous.get(iso)

In [ ]:
# Output complete JSON file
with open(jsonOutputFilePath, 'w', encoding='utf-8') as jsonFile:
    jsonFile.write(json.dumps(dict_features, indent=4))

## Scatter plot data joins and outputs

In [13]:
# Define join datasets for scatter plot json file
df_gdpScatter = out_gdp[0]['percentile'].reset_index()
df_medalsScatter = df_medalsPerCapitaMill['percentile'].reset_index()

In [14]:
# Merge dataset for json file transformation, drop missing data
df_scatter = df_gdpScatter.merge(df_medalsScatter,
                                 how='inner',
                                 on='nationality')\
                          .rename({'percentile_x': 'percGDP', 'percentile_y': 'percMedals'}, axis=1)\
                          .set_index('nationality')

In [15]:
# Parse dictionary for json output
list_scatterGDP = list(df_scatter.to_dict(orient='dict')['percGDP'].values())
list_scatterMedals = list(df_scatter.to_dict(orient='dict')['percMedals'].values())

In [ ]:
# Output JSON files for scatterGDP array
with open(r'C:\Users\billy\group_project\Dashboard\static\js\data\chartData\list_scatterGDP.json', 'w', encoding='utf-8') as jsonFile:
    jsonFile.write(json.dumps(list_scatterGDP, indent=4))

In [ ]:
# Output JSON files for scatterMedals array
with open(r'C:\Users\billy\group_project\Dashboard\static\js\data\chartData\list_scatterMedals.json', 'w', encoding='utf-8') as jsonFile:
    jsonFile.write(json.dumps(list_scatterMedals, indent=4))